In [1]:
"""
Loading important package of spark
"""
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark import SparkContext
import pyspark.sql.functions as F
from pyspark.sql.functions import *

In [2]:
"""
Spark session creater
"""
st = SparkSession \
        .builder \
        .appName('ChallengeML') \
        .getOrCreate()

In [3]:
"""
Load data function for loading data..
@param -
        path - path of file
        header_value - header value, incase true first row will be header

@return - dataframe of loaded intended data.
"""

def load_data(path,header_value):
  df = st.read.csv(path,inferSchema=True,header=header_value)
  return df

In [4]:
df = load_data('/Users/eduardobertaud/Desktop/challengeML/rating.csv',True)

In [5]:
"""
Functions to get nb_previous_ratings and avg_ratings_previous..
@param -
        dataFrame - Data frame with rates

@return - dataframe of loaded intended data.
"""
def nb_previous_ratings(df):
    df=df.withColumn("nb_previous_ratings",rank().over(Window.partitionBy("userId").orderBy("timestamp")) -1 )
    return df

def avg_ratings_previous(df):
  WindowSpec = Window.partitionBy("userId").rowsBetween(Window.unboundedPreceding,-1)

  df = df.withColumn("avg_ratings_previous", F.avg(F.col("rating")).over(WindowSpec.orderBy("userId", "timestamp")))
  return df

In [6]:
"""
calling Function nb_previous_rating and assign to dataFrame
"""
df = nb_previous_ratings(df)

"""
calling Function nb_previous_rating and assign to dataFrame
"""
df = avg_ratings_previous(df)

In [7]:
"""
Creating a new parquet file with the changes created
"""
df.write.mode('overwrite').parquet("/Users/eduardobertaud/Desktop/challengeML/new_ratings.parquet")

In [8]:
"""
Stop the spark session
"""
#st.stop()

'\nStop the spark session\n'